In [0]:
try:
    import gdown
    print("gdown is installed.")
except ImportError:
    print("gdown is not installed. Installing it now...")
    try:
        import subprocess
        subprocess.check_call(["pip", "install", "gdown"])
        print("gdown has been successfully installed.")
        import gdown  # Importing again to use it after installation
    except Exception as e:
        print(f"Error installing gdown: {e}")


from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.sql.functions import when, col

import pandas as pd
import numpy as np
import time
from datetime import datetime


try:
    from xgboost.spark import SparkXGBClassifier
    print("sparkxgb is installed.")
except ImportError:
    print("sparkxgb is not installed. Installing it now...")
    try:
        import subprocess
        command = "pip install pyarrow pandas venv-pack xgboost"
        subprocess.run(command, shell=True, check=True)
        print("sparkxgb has been successfully installed.")
        from xgboost.spark import SparkXGBClassifier
    except Exception as e:
        print(f"Error installing sparkxgb: {e}")

from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import FloatType, IntegerType
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



gdown is not installed. Installing it now...
gdown has been successfully installed.


sparkxgb is not installed. Installing it now...
sparkxgb has been successfully installed.


In [0]:
from pyspark.ml import PipelineModel
model_path = r"file:///databricks/driver/predictive_model"
# Load the model from the saved folder
loaded_model = PipelineModel.load(model_path)

In [0]:
csv_path = r"file:/databricks/driver/tmp/static/random_chunk.csv"
df = spark.read.option("header", "true").option("inferschema", "true").csv(csv_path)
df.display()
df.schema

_c0,uid,id_orig_p,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Malicious
0,CxRJSN1zDC8dmbgdb3,49708,6667,tcp,-,0.0,0,0,S0,S,1,60,0,0,1


Out[3]: StructType([StructField('_c0', IntegerType(), True), StructField('uid', StringType(), True), StructField('id_orig_p', IntegerType(), True), StructField('id_resp_p', IntegerType(), True), StructField('proto', StringType(), True), StructField('service', StringType(), True), StructField('duration', DoubleType(), True), StructField('orig_bytes', IntegerType(), True), StructField('resp_bytes', IntegerType(), True), StructField('conn_state', StringType(), True), StructField('history', StringType(), True), StructField('orig_pkts', IntegerType(), True), StructField('orig_ip_bytes', IntegerType(), True), StructField('resp_pkts', IntegerType(), True), StructField('resp_ip_bytes', IntegerType(), True), StructField('Malicious', IntegerType(), True)])

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.functions import expr
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType


# Create a Spark session
spark = SparkSession.builder.appName("StreamingExample").getOrCreate()


csv_path = r"file:/databricks/driver/tmp/"
streaming_df = spark.readStream.option("header", "true").schema(df.schema).option("maxFilesPerTrigger", 1) .csv(csv_path)


try:
    predictions_df = loaded_model.transform(streaming_df)
    # Add additional processing logic here if needed

    # Define the sink for the streaming output
    predictions_query = predictions_df.writeStream \
        .format("memory") \
        .queryName("predictions") \
        .outputMode("append") \
        .start()

    #predictions_query.awaitTermination()

except:
    print(f"Error in streaming query")



In [0]:
%%bash
rm -r PredictionData
mkdir PredictionData

rm: cannot remove 'PredictionData': No such file or directory


In [0]:
%%bash
rm -r processing_aid
# Replace 'YOUR_FILE_ID' with the actual file ID from the Google Drive link
file_id="1eARwwYFGSHpaOcuA8GxwwvzMeOgFvafD"

# Replace 'output_file.zip' with the desired output file name
output_file="simulated_actions.csv"

# Download the file from Google Drive using gdown
gdown "https://drive.google.com/uc?id=${file_id}" -O "${output_file}"

rm: cannot remove 'processing_aid': No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1eARwwYFGSHpaOcuA8GxwwvzMeOgFvafD
To: /databricks/driver/simulated_actions.csv
100%|██████████| 6.51k/6.51k [00:00<00:00, 5.56MB/s]


In [0]:
import random
def prediction_processor(df, existing_data = "none"):
    simulated_action_data = pd.read_csv("simulated_actions.csv")
    if len(df) >1:
        if not isinstance(existing_data, pd.DataFrame):
            df["ID"] = ""
            df["TimeStamp"] = ""
        else:
            df = df.merge(existing_data[["IOT Connection ID", "ID", "TimeStamp"]], how='left', on='IOT Connection ID' )
            df[["ID", "TimeStamp"]] = df[["ID", "TimeStamp"]].fillna("")
        for index,row in df.iterrows():
            if row["ID"] == "" and row["Prediction"] == "Malicious":
                df.loc[index, "ID"] = random.randint(51, 100)
            if row["ID"] == "" and row["Prediction"] == "Benign":
                df.loc[index, "ID"] = random.randint(1, 50)
            if row["TimeStamp"] == "":
                df.loc[index, "TimeStamp"] = datetime.now()
        df = df.merge(simulated_action_data, how='left', on='ID' )
        print(df.columns)
        return df[["TimeStamp", "IOT Connection ID" , "ID", "County", "Device IP", "Device Type", "Malware Attack Attempt", "Recommended Action"]]
    else:
        print("No Predictions made yet")




In [0]:

existing_data = "none"
while True:
  print(datetime.now())
  temp_df = spark.sql("select uid, prediction from predictions").toPandas()
  if len(temp_df) > 1:
    new_column_names = {'uid': 'IOT Connection ID', 'prediction': 'Prediction'}
    temp_df.rename(columns=new_column_names, inplace=True)
    replacement_mapping = {1: 'Malicious', 0: 'Benign'}
    temp_df['Prediction'] = temp_df['Prediction'].replace(replacement_mapping)
    output = prediction_processor(temp_df, existing_data)
    existing_data = output.copy()
    output.to_csv(r"/databricks/driver/PredictionData/PredictionData.csv")
    print(output)
  time.sleep(2)

2023-12-02 16:04:39.103602
Index(['IOT Connection ID', 'Prediction', 'ID', 'TimeStamp', 'County',
       'Device IP', 'Device Type', 'Malware Attack Attempt',
       'Recommended Action'],
      dtype='object')
                     TimeStamp   IOT Connection ID  ID      County  \
0   2023-12-02 16:04:39.299325  C656YD2TqZazmCySNe  46      Carver   
1   2023-12-02 16:04:39.299743  COw1bC4rVHzw5SjqS8  79      Ramsey   
2   2023-12-02 16:04:39.300078   CLU5zk4m7xRPVamzp  71  Washington   
3   2023-12-02 16:04:39.300967  CDcMlk2RbJ5mBmOGOb  72      Wright   
4   2023-12-02 16:04:39.301465  Co6tr44hTakRFGmve7  67      Isanti   
5   2023-12-02 16:04:39.301845  CsY0wK2W28YxIxA7Td  75     Chisago   
6   2023-12-02 16:04:39.302205  CWZwBr3e7PEdcjQ047  58       Scott   
7   2023-12-02 16:04:39.302534   CqZ9WP3LWmxKQtXZ3  55    Hennepin   
8   2023-12-02 16:04:39.302950  Cf2llj1vk5kImgnlJi  82  Washington   
9   2023-12-02 16:04:39.303309  CNRaQS1qAUGftcX0o1  30       Scott   
10  2023-12-02 16:0